Objectifs : 
- Récupérer les albums les plus vendus 
- Récupérer les prix associés
- Récupérer les informations complémentaires

Nous prenons le plus gros marché du disque existant à savoir les Etats-Unis : [https://www.riaa.com/gold-platinum/?tab_active=top_tallies&ttt=T1A#search_section]

In [2]:
import scrapy
from scrapy import Request

In [21]:
class RIAASpider(scrapy.Spider):
    name = "riaa"
    allowed_domains = ["www.riaa.com"]
    start_urls = ['https://www.riaa.com/gold-platinum/?tab_active=awards_by_album#search_section']

    def parse(self, response):
            import re
            cpt = 0
            for album in response.xpath('//tr[@class="table_award_row"]'):
                if cpt != 10:
                    artist = album.xpath('td[@class="artists_cell"]/text()').extract_first()
                    info = album.xpath('td[@class="others_cell"]/text()').extract()
                    album_title = info[0]
                    label = info[1]
                    certif_UT = album.xpath('td[@class="others_cell format_cell"]/text()').extract_first()
                    certif_UT = re.findall(r"\r\n                    (.+?)                    ",certif_UT)[0]
                    cpt += 1 
                    yield { "Artist : " : artist, "Album : " : album_title, "Label : " : label, "Certification unit" : certif_UT}

In [3]:
class SNEPSpider(scrapy.Spider):
    name = "snep"
    allowed_domains = ["www.snepmusique.com"]
    start_urls = ['https://snepmusique.com/les-certifications/?certification=Double%20Diamant,Triple%20Diamant,Quadruple%20Diamant&categorie=Albums']

    def parse(self, response):
            import re
            cpt = 0
            for album in response.xpath('//div[@class="description"]'): 
                if cpt != 10:
                    artist = album.xpath('div[@class="artiste"]/text()').extract_first()
                    album_title = album.xpath('div[@class="titre"]/text()').extract_first()
                    label = album.xpath('div[@class="editeur"]/text()').extract_first()
                    cpt += 1 
                    yield { "Artist : " : artist, "Album : " : album_title, "Label : " : label}
            
            cpt = 0
            for album in response.xpath("//div[contains(@class, 'certif icon')]"):
                if cpt != 10 :
                    certif_UT = album.css("div::text").extract_first()
                    cpt += 1

                    yield {"Certification unit" : certif_UT}

In [ ]:
import scrapy
from scrapy import Request

class labelspider(scrapy.Spider):
    name="label"
    allowed_domains = ["www.discogs.com"]
    start_urls = ['https://www.discogs.com/fr/Taylor-Swift-1989/master/750386']

    def parse(self, response):
        import re
        for label in response.xpath("//tr[contains(@class, 'card r_tr main release')]"):
        #for label in response.xpath('//div[@class="section m_versions toggle_section toggle_section_remember toggle_section_collapsed"/p/a[@class="section_content toggle_section_content"/p/a[@class="cards table_responsive layout_text-only"]'):
            label_item = label.css("tr::text").extract_first()
            #label_item = label.css("td[@class='label has_header']::text").extract_first()
            yield label_item

In [1]:
import pandas as pd
import pymongo

In [21]:
result = [{'album_title': 'CRACKED REAR VIEW',
 'artist': 'HOOTIE & THE BLOWFISH',
 'certif_UT': '21000000',
 'label': 'RHINO'}, {'album_title': 'CRACKED REAR VIEW',
 'artist': 'HOOTIE & THE BLOWFISH',
 'certif_UT': '21000000',
 'label': 'RHINO'}]

In [34]:
client = pymongo.MongoClient('mongodb://localhost:27017/')
database = client['test']
collection = database['test']

In [35]:
df = pd.DataFrame(result)

In [36]:
DOCUMENTS = df.to_dict('records')

In [37]:
collection.insert_many(result)

ServerSelectionTimeoutError: localhost:27017: [Errno 61] Connection refused, Timeout: 30s, Topology Description: <TopologyDescription id: 5fe8a725dd588de5f00cb9a2, topology_type: Single, servers: [<ServerDescription ('localhost', 27017) server_type: Unknown, rtt: None, error=AutoReconnect('localhost:27017: [Errno 61] Connection refused')>]>